In [156]:
import numpy as np #Importing packages
from numpy.linalg import matrix_power

In [165]:
#Variables and constants
N=2
p=5
e=np.e
pi=np.pi
K11, K12, K21, K22=1, 1, 1, 1
omega=e**(2*pi*1j/p)

In [166]:
#Important matrix methods

def cosine(angle):
    return np.cos(angle)
def id(p):
    return np.identity(p)
def id_chain(n, p):
    return np.identity(p**n)
print(id(1))

[[1.]]


In [167]:
def matrix_chain(n_identity_anterior, hamiltonian, n_identity_posterior):
    if n_identity_anterior!=0 and n_identity_posterior!=0:
        first_compute=np.tensordot(id_chain(n_identity_anterior, p), hamiltonian, axes=0) 
        return np.tensordot(first_compute, id_chain(n_identity_anterior, p), axes=0)
    elif n_identity_anterior==0:
        return np.tensordot(hamiltonian,id_chain(n_identity_posterior, p) , axes=0)
    else:
        return np.tensordot(id_chain(n_identity_anterior, p), hamiltonian , axes=0)

In [168]:
def umaker(p):
    matrix=np.zeros((p, p), dtype=complex)
    for i in range(p):
        matrix[i][i]=omega**i
    return matrix
    
def vmaker(p):
    matrix=id(p)
    for i in range(p):
        matrix[i]=(np.append(matrix[i][p-1], matrix[i][:p-1]))
    return matrix
print(umaker(p))

[[ 1.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.30901699+0.95105652j  0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.80901699+0.58778525j
   0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.80901699-0.58778525j  0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.30901699-0.95105652j]]


In [169]:
def hamiltonian(n, p):
    matrix=np.zeros((p**n, p**n), dtype=complex)
    for site in range(n-1):
        localU=np.zeros((p**n, p**n), dtype=complex)
        for spin in range(p):
            m1=matrix_chain(site+1, umaker(p), n-site-2)
            m2=matrix_chain(site, np.transpose(np.conjugate(umaker(p))), n-site-1)
            Uchain=np.reshape(matrix_power(np.multiply(m1, m2), spin), (p**n, p**n))
            localU+=((K11+2*K12*cosine(2*pi*spin/p))*Uchain)
        matrix+=localU
    for site in range(n):
        localV=np.zeros((p**n, p**n), dtype=complex)
        for spin in range(p):
            Vchain=np.reshape(matrix_power((matrix_chain(site, vmaker(p), n-site-1)), spin), (p**n, p**n))
            localV+=((K21+2*K22)*Vchain)
        matrix+=localV
    return matrix

In [170]:
matrix=hamiltonian(N, p)

In [171]:
print(np.linalg.eig(matrix)[0])

[ 8.65800281e+01-3.18933999e-31j -4.15800281e+01+1.19899352e-30j
  8.09016994e+00+1.18697057e-30j  8.09016994e+00+3.15267970e-16j
 -3.09016994e+00+3.88578059e-16j -3.09016994e+00-9.36750677e-17j
 -4.75287269e-15+1.93189081e-16j  5.21154810e-16-1.62776001e-15j
  4.26152329e-16+1.57333866e-15j -5.84962040e-16+1.36412353e-17j
 -2.94327162e-17+1.49179149e-16j  9.85916634e-16-3.67565381e-17j
 -2.52582248e-32-1.10832232e-31j  3.66475919e-32+3.36823700e-32j
 -3.83308236e-32-4.10806980e-33j  1.36897791e-32-2.42749447e-32j
  3.79746924e-47+2.03344375e-48j -9.57983734e-48-8.36039800e-48j
 -4.74176976e-49+2.28136314e-48j -4.26036903e-49+5.09399879e-49j
 -1.96067439e-50-1.02838067e-50j  4.11821637e-65+5.87893697e-65j
 -1.63079054e-66-1.44619759e-67j  6.47477688e-83-1.23671813e-82j
 -3.67655501e-98+4.33927033e-98j]
